<a href="https://colab.research.google.com/github/d-s-k-ai/ML_projects/blob/main/Fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from tqdm import tqdm
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [20]:
true = pd.read_csv("/content/True.csv")
fake = pd.read_csv("/content/Fake.csv")

In [21]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [22]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [23]:
true['label'] = 1
fake['label']=0

In [24]:
true.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


To concat the true and fake news dataset into data

In [25]:
data = pd.concat([true,fake], axis = 0)
data.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


*resuffle the data to train the data more accurately*

In [26]:
data =data.sample(frac=1)
data.reset_index(inplace=True)

In [27]:
data.drop(['index'],axis = 1, inplace = True)

In [28]:
data.head()

,title,text,subject,date,label
0,CUBANS BRUTALLY ATTACK Brave Man Waving US Fla...,This is what having no freedom of speech looks...,Government News,"May 2, 2017",0
1,Rights group files legal request on Trump busi...,WASHINGTON (Reuters) - The American Civil Libe...,politicsNews,"January 20, 2017",1
2,THREE RESIGNATIONS AT CNN After Botched Trump-...,CNN employees have resigned for their role in ...,left-news,"Jun 26, 2017",0
3,Speed over safety? China's food delivery indus...,BEIJING (Reuters) - A scooter driver in a brig...,worldnews,"September 28, 2017",1
4,Micheal Moore Demands MI Republican Gov. Be A...,Michael Moore has joined others in calling for...,News,"January 6, 2016",0


In [29]:
data.drop(['title','subject','date'],axis = 1)

,text,label
0,This is what having no freedom of speech looks...,0
1,WASHINGTON (Reuters) - The American Civil Libe...,1
2,CNN employees have resigned for their role in ...,0
3,BEIJING (Reuters) - A scooter driver in a brig...,1
4,Michael Moore has joined others in calling for...,0
...,...,...
44893,KAMPALA (Reuters) - Uganda is launching a driv...,1
44894,What happens when a black police officer sings...,0
44895,WASHINGTON (Reuters) - A proposal by a U.S. Se...,1
44896,UNITED NATIONS (Reuters) - United Nations poli...,1


Now we will convert the alphabetical data to numerical data using Vectorizer

In [30]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [31]:
# to alter the words to its root word (stemmer)

def stemming(text):
  text = re.sub('[^a-zA-Z]',' ',text)
  text = text.lower()
  text = re.sub(r'https?://\S+|www\.\S+','',text)     #remove urls and hyperlinks
  text = re.sub(r'<.*?>','',text)                     #remove HTML tags
  text = re.sub(r'\n','',text)                        #remove newline characters
  text = re.sub(r'[^\w\s]','',text)                   #remove punctuation marks and special characters
  text = re.sub(r'\d','',text)                        #remove digits
  return text



In [32]:
from tqdm import tqdm
tqdm.pandas()
data["text"] = data["text"].progress_apply(stemming)

100%|██████████| 44898/44898 [00:13<00:00, 3443.59it/s]


In [33]:
data["text"]

,text
0,this is what having no freedom of speech looks...
1,washington reuters the american civil libe...
2,cnn employees have resigned for their role in ...
3,beijing reuters a scooter driver in a brig...
4,michael moore has joined others in calling for...
...,...
44893,kampala reuters uganda is launching a driv...
44894,what happens when a black police officer sings...
44895,washington reuters a proposal by a u s se...
44896,united nations reuters united nations poli...


In [34]:
x = data["text"]
y = data["label"]

In [35]:
x

,text
0,this is what having no freedom of speech looks...
1,washington reuters the american civil libe...
2,cnn employees have resigned for their role in ...
3,beijing reuters a scooter driver in a brig...
4,michael moore has joined others in calling for...
...,...
44893,kampala reuters uganda is launching a driv...
44894,what happens when a black police officer sings...
44895,washington reuters a proposal by a u s se...
44896,united nations reuters united nations poli...


In [36]:
y

,label
0,0
1,1
2,0
3,1
4,0
...,...
44893,1
44894,0
44895,1
44896,1


In [37]:
# splitting the data

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state =45)

print("x_train=",x_train.shape)
print("y_train",y_train.shape)
print("x_test",x_test.shape)
print("y_test",y_test.shape)

x_train= (35918,)
y_train (35918,)
x_test (8980,)
y_test (8980,)


In [38]:
# to performering verctorizer steps to convert the categorical data to numerical data

from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [39]:
#lets check the vectorizer

print(xv_train)

  (0, 101045)	0.03998900754224772
  (0, 91958)	0.017869490428830345
  (0, 23505)	0.050478963188494916
  (0, 90564)	0.04000574248658759
  (0, 53732)	0.04361339861338414
  (0, 43155)	0.08338186552190446
  (0, 91991)	0.1918615719315012
  (0, 94022)	0.3384591386584739
  (0, 8765)	0.04072103679085282
  (0, 52200)	0.16125465997368635
  (0, 3460)	0.05004168972293246
  (0, 15282)	0.16754730308924187
  (0, 60584)	0.3225093199473727
  (0, 45118)	0.037542832067995975
  (0, 83005)	0.07537884828080949
  (0, 97593)	0.029339711474012254
  (0, 86268)	0.03241606714173923
  (0, 17700)	0.3135799172615093
  (0, 32660)	0.03663453013100315
  (0, 102426)	0.05249972920863622
  (0, 94270)	0.08318443908100408
  (0, 2765)	0.07019496494566135
  (0, 60346)	0.09869083815590698
  (0, 102134)	0.038952135887939776
  (0, 15487)	0.055794033106788266
  :	:
  (35917, 87239)	0.027520777522983143
  (35917, 5367)	0.029779134427774124
  (35917, 1112)	0.02975637216682011
  (35917, 1982)	0.08471908825103212
  (35917, 10328)	0.0

In [40]:
#lets train the model

#using logistic regression model

from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train,y_train)

pred_LR = LR.predict(xv_test)

LR.score(xv_test,y_test)

from sklearn.metrics import classification_report
print (classification_report(y_test,pred_LR))


              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4629
           1       0.98      0.99      0.99      4351

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [41]:
xv_train_predictions = LR.predict(xv_train)
training_data_accuracy = accuracy_score(xv_train_predictions,y_train)
print('accuracy of the training data',training_data_accuracy)

xv_test_predcitions = LR.predict(xv_test)
testing_data_accuracy = accuracy_score(xv_test_predcitions,y_test)
print('accuracy of the testing data',testing_data_accuracy)

accuracy of the training data 0.9916198006570521
accuracy of the testing data 0.9868596881959911


In [42]:
# # lets try some other models

# #using XGboost

# import xgboost as xgb

# #create a DMatrix for XG boost

# dtrain = xgb.DMatrix(xv_train,label = y_train)
# dtest = xgb.DMatrix(xv_test,label = y_test)

# params = {
#     'objective':'binary:logistic',   # For binary classification
#     'max_depth': 6,                # max number of trees
#     'eta'      : 0.3,              # Learning Rate
#     'eval_metric': 'logloss'         # Evaluation metric
# }


# num_round = 100  # Number of boosting rounds
# evallist = [(dtest, 'eval'), (dtrain, 'train')]

# # FITTING THE MODEL
# xgb_model = xgb.train(params, dtrain, num_round, evallist, early_stopping_rounds=10)

# #PREDICTING THE MODEL

# y_pred = xgb_model.predict(dtest)
# y_pred = [1 if pred >0.5 else 0 for pred in y_pred]   # for binary classification

# accuracy = accuracy_score(y_test, y_pred)
# print(f'Accuracy: {accuracy}')

# print ("classification_report of Xgboost")
# print(classification_report(y_test,y_pred))
# print("accuracy of the testing data",accuracy)



In [43]:
# # using xgboost from sklearn library

from xgboost import XGBClassifier

xgbs_model = XGBClassifier()
xgbs_model.fit(xv_train,y_train)

pred_xgbs = xgbs_model.predict(xv_test)

xgbs_model.score(xv_test,y_test)



print(classification_report(y_test,pred_xgbs))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4629
           1       1.00      1.00      1.00      4351

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



In [44]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier()
RF.fit(xv_train,y_train)

pred_RF = RF.predict(xv_test)

RF.score(xv_test,y_test)

print(classification_report(y_test,pred_RF))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4629
           1       0.99      0.99      0.99      4351

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [45]:
from sklearn.ensemble import GradientBoostingClassifier

GB = GradientBoostingClassifier()
GB.fit(xv_train,y_train)

pred_GB = GB.predict(xv_test)

GB.score(xv_test,y_test)

print(classification_report(y_test,pred_GB))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      4629
           1       0.99      1.00      1.00      4351

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



In [46]:
# create a predicted model to check the model

def output_label(n):
  if n==0:
    return "it is a Fake News"
  elif n==1:
    return "it is a True News"

In [47]:
# create a manual testing function
def manual_testing(news):
  testing_news = {"text":[news]}
  new_def_test = pd.DataFrame(testing_news)
  new_def_test["text"] = new_def_test["text"].apply(stemming)
  new_x_test = new_def_test["text"]
  new_xv_test= vectorization.transform(new_x_test)

  pred_LR = LR.predict(new_xv_test)
  pred_RF = RF.predict(new_xv_test)
  pred_GB = GB.predict(new_xv_test)
  pred_xgbs = xgbs_model.predict(new_xv_test)


  return"\n\nLR Prediction:{}  \n\nRF Predcition:{} \n\nGB Prediction:{}  \n\nxgbs Prediction:{}".format(
      output_label(pred_LR[0]), output_label(pred_RF[0]) ,output_label(pred_GB[0]) ,output_label(pred_xgbs[0]))


In [48]:
news_article = str(input())

Many people have raised the alarm regarding the fact that Donald Trump is dangerously close to becoming an autocrat. The thing is, democracies become autocracies right under the people s noses, because they can often look like democracies in the beginning phases. This was explained by Republican David Frum just a couple of months into Donald Trump s presidency, in a piece in The Atlantic called  How to Build an Autocracy. In fact, if you really look critically at what is happening right now   the systematic discrediting of vital institutions such as the free press and the Federal Bureau of Investigation as well the direct weaponization of the Department of Justice in order to go after Trump s former political opponent, 2016 Democratic nominee Hillary Clinton, and you have the makings of an autocracy. We are more than well on our way. Further, one chamber of Congress, the House of Representatives, already has a rogue band of Republicans who are running a parallel investigation to the of

In [49]:
manual_testing(news_article)

'\n\nLR Prediction:it is a Fake News  \n\nRF Predcition:it is a Fake News \n\nGB Prediction:it is a Fake News  \n\nxgbs Prediction:it is a Fake News'

In [53]:
news_article = str(input())

WASHINGTON (Reuters) - Trump campaign adviser George Papadopoulos told an Australian diplomat in May 2016 that Russia had political dirt on Democratic presidential candidate Hillary Clinton, the New York Times reported on Saturday. The conversation between Papadopoulos and the diplomat, Alexander Downer, in London was a driving factor behind the FBIâ€™s decision to open a counter-intelligence investigation of Moscowâ€™s contacts with the Trump campaign, the Times reported. Two months after the meeting, Australian officials passed the information that came from Papadopoulos to their American counterparts when leaked Democratic emails began appearing online, according to the newspaper, which cited four current and former U.S. and foreign officials. Besides the information from the Australians, the probe by the Federal Bureau of Investigation was also propelled by intelligence from other friendly governments, including the British and Dutch, the Times said. Papadopoulos, a Chicago-based i

In [54]:
manual_testing(news_article)

'\n\nLR Prediction:it is a True News  \n\nRF Predcition:it is a True News \n\nGB Prediction:it is a True News  \n\nxgbs Prediction:it is a True News'

**To take a link as input**

In [50]:
# !pip install newspaper3k




In [51]:
# !apt install newspaper3k

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package newspaper3k


In [52]:
# from newspaper import Article

ImportError: lxml.html.clean module is now a separate project lxml_html_clean.
Install lxml[html_clean] or lxml_html_clean directly.

In [ ]:
# def fetch_article_from_url(url):
#   try:
#     article = Article(url)
#     article.download()
#     article.parse()
#     return article.text
#   except Exception as e:
#     print(f"error fetching the article:{e}")
#     return None

In [ ]:
# url = input('enter the url of the article')
# news_article = fetch_article_from_url(url)

# if news_article:
#   prediction = manual_testing(news_article)
#   print("the article is",prediction)

# else:
#   print("unable to fetch the article")

In [ ]:
import pickle

model_LR = LR
model_RF = RF
model_GB = GB
model_xgbs = xgbs_model


models ={
    "LogisticRegression":model_LR,
    "RandomForestClassifier":model_RF,
    "GradientBoostingClassifier":model_GB,
    "XGBClassifier": model_xgbs
}

#saving the dictionary of model to pickle

with open("models.pkl","wb") as file:
  pickle.dump(models,file)


In [ ]:
# Load the dictionary of models
with open('models.pkl', 'rb') as f:
    models = pickle.load(f)

# Access each model from the dictionary
model_LR = models['LogisticRegression']
model_RF = models['RandomForestClassifier']
model_GB = models['GradientBoostingClassifier']
model_XGB = models['XGBClassifier']